<a href="https://colab.research.google.com/github/ramm1024/trabajotitulo/blob/main/POC_TT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai
!pip install -q pattern

In [277]:
from openai import OpenAI
import os
from google.colab import userdata
from google.colab import drive
import pandas as pd
import time
import csv
from io import StringIO
from pattern.text.en import singularize
drive.mount('/content/drive')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KE')
client = OpenAI()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [278]:
def set_model(model_choice:str)-> str:
  #https://openai.com/api/pricing/
  if model_choice == "3.5t":
    return "gpt-3.5-turbo"
  elif model_choice == "4":
    return "gpt-4"
  elif model_choice == "4t":
    return  "gpt-4-turbo"
  elif model_choice == "3.5t0125":
    return "gpt-3.5-turbo-instruct"
  elif model_choice == "4o":
    return "gpt-4o"
  elif model_choice == "3.5t0125":
    return "gpt-3.5-turbo-0125"
  else:
    return "gpt-3.5-turbo-0125"#"text-embedding-3-large"
model_choice = set_model("4o")

In [ ]:
regulation.df

In [363]:
#input df: df['columna']
#output str: string concatenado por ','
def get_data(df:pd.DataFrame)->str:
  return ','.join(map(str,df.tolist()))

#input data: df con datos a guardar en drive
#output: guarda .csv en drive
def save_output(data:pd.DataFrame):
  with open('/content/drive/MyDrive/11vo/poctestdata/out_'+dataset_filename, 'w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['data_type', 'column_name'])
    for item in data:
      writer.writerow([item,','.join(data[item])])

def sanitize(data_type:str)->str:
  string = singularize(data_type.lower())
  if string == 'email addres':
    return 'email address'
  elif string == 'addres':
    return 'address'
  else:
    return string

def upload_policy(policy_string, filename):
  with open('/content/drive/MyDrive/11vo/poctestdata/' + filename + '.txt', 'w') as f:
    f.write(policy_string)

#input data: str con datos unidos por ;
#output tipo_datos: lista de diccionarios con el tipo de dato encontrado y la cantidad de instancias
#def send_prompt(data:str, model_choice:int)->str:
def send_prompt(mode:str='get_data_types', data:str='', regulation:str='HIPAA', lang:str='english')->str:
  completion = client.chat.completions.create(
    model=model_choice,
    messages=set_prompt(mode=mode,data=data,regulation=regulation,lang=lang)
  )
  return completion.choices[0].message.content

def translate(string:str, lang:str)->str:
  return send_prompt(mode='translate',data=string,lang=lang)

#input data: str con los datos de una columna unidos por ','
#output messages: msg a enviar como prompt para analisis
def set_prompt(mode:str, data:str, regulation:str, lang:str)->str:
  messages = []
  if mode == 'get_data_types':
    messages=[
                      {"role": "system", "content": "You are a data parser"},
                      {"role": "system", "content": "Your datasets come in form of a string separated by semicolons"},
                      {"role": "system", "content": "Yopu will use the dataset to infer the type of personal information that it contains"},
                      #{"role": "system", "content": "The types of data that it can contain are Personal Information, Sensitive Information, PHI, HIPAA protected information and non personal information"},
                      {"role": "system", "content": f"Return only the name of the type of personal information"},
                      #{"role": "system", "content": f"All the content must be written in {lang}"},
                      {"role": "user", "content": f"{data}"},
    ]
  elif mode == 'get_regulations':
    messages=[
                      {"role": "system", "content": "You are a data redactor that creates policies based on a specific regulation"},
                      {"role": "system", "content": f"I need to make a table that describes the rules and guidelines needed to comply with the regulation {regulation}"},
                      {"role": "system", "content": f"Extract all the rules and guidelines found"},
                      {"role": "system", "content": "Adapt this information to a format that follows exactly: Key Points, Description, Compliance Requirements"},
                      {"role": "system", "content": "Include rows specific to the data access rights of said regulation"},
                      {"role": "system", "content": "This table should be as extense as needed and properly separate each function in different rows"},
                      {"role": "system", "content": f"You output will be only text tha represents a csv separated by semicolons"},
                      {"role": "system", "content": f"You are only allowed to print a string with a csv that has the headers: Key Points, Description, Compliance Requirements"},
                      {"role": "user", "content": "Print the output csv string"},
    ]
  elif mode == 'translate':
      messages=[
          {"role": "system", "content": "You are a text translator"},
          {"role": "system", "content": f"Translate the following text to {lang}"},
          {"role": "user", "content": f"{data}"},
    ]
  else:
      raise ValueError(f"Invalid mode: {mode}")
  return messages
"""
#input df: csv con informacion personal
#output data_types: tipos de datos sensibles encontrados en cada columna
def get_data_types(df:pd.DataFrame)->dict[list]:
  data_types = dict()
  for column_name in df.columns:
    data_type = send_prompt(data=get_data(df[column_name]))
    sanitized_data_type = sanitize(data_type)
    if sanitized_data_type not in data_types:
      data_types[sanitized_data_type] = [column_name]
    else:
      data_types[sanitized_data_type].append(column_name)
  return data_types

#input regulation: Name of the regulation to use
#output df_cumplimiento: df[Key Points, Description, Compliance Requirements]
def get_cumplimiento(regulation:str)->pd.DataFrame:
  csv = send_prompt(mode='get_regulations',regulation=regulation)
  df_cumplimiento = pd.read_csv(StringIO(csv),sep=";",header=1)
  for column_name in df_cumplimiento.columns:
    df_cumplimiento = df_cumplimiento.rename(columns={df_cumplimiento.columns[0]: 'Key Points', df_cumplimiento.columns[1]: 'Description', df_cumplimiento.columns[2]:'Compliance Requirements'}, inplace=False)
  return df_cumplimiento

def generate_privacy_policy(df_cumplimiento:pd.DataFrame, df_dataset:pd.DataFrame, lang:str)->str:

    Generate a data privacy policy from a DataFrame containing privacy regulations.

    Parameters:
    df_cumplimiento (pd.DataFrame): DataFrame with columns like 'Regulation', 'Description', 'Key Points', 'Compliance Requirements'.
    df_dataset (pd.DataFrame): DataFrame with personal information.
    lang (str): Language code for translation.

    Returns:
    str: A formatted data privacy policy.

    # Check for necessary columns in the DataFrame
    required_columns = ['Description', 'Key Points', 'Compliance Requirements']
    if not all(column in df_cumplimiento.columns for column in required_columns):
        raise ValueError(f"DataFrame must contain the following columns: {required_columns}")

    # Start with an introductory section
    policy = "Data Privacy Policy\n"
    policy += "=" * 20 + "\n\n"
    policy += "This policy outlines our commitment to protecting personal data and complying with data privacy regulations.\n\n"

    # Generate sections for each regulation
    for index, row in df_cumplimiento.iterrows():
      if pd.isnull(row['Compliance Requirements']) and pd.isnull(row['Description']):
        policy += f"\nControl a realizar: {row['Key Points']}\n"
      else:
        if row['Key Points'].lower() != 'nan': policy += f"Key Point: {row['Key Points']}\n"
        if row['Compliance Requirements'].lower() != 'nan': policy += f"Compliance Requirements: {row['Compliance Requirements']}\n"
        if row['Description'].lower() != 'nan': policy += f"Description:\n{row['Description']}\n"
      policy += "\n" + "-" * 20 + "\n"

    policy += "Contact Information:\n"
    policy += "If you have any questions about this policy, please contact us at [your contact information].\n\n"

    policy += "Policy Review and Updates:\n"
    policy += "We may update this policy from time to time to reflect changes in our practices or legal requirements. Please check back periodically for updates.\n"

    #omegalul
    policy += policy_data_types(df_cumplimiento,df_dataset)
    if lang != 'english':
      policy = translate(policy,lang)
    return policy

def policy_data_types(df_cumplimiento:pd.DataFrame,df_dataset:pd.DataFrame)->str:

    Generate a data policy based on personal or sensitive information data types from a CSV file.

    Parameters:
    df_cumplimiento
    df_dataset

    Returns:
    str: A formatted data policy.

    # Start generating the policy
    policy = "Data Privacy Policy\n"
    policy += "=" * 20 + "\n\n"
    policy += "This policy outlines our commitment to protecting personal and sensitive data types and complying with data privacy regulations.\n\n"

    # Add sections based on the data types
    policy += "Data Types Covered\n"
    policy += "-" * 20 + "\n"
    for data_type in get_data_types(df_dataset):
        policy += f"- {data_type}\n"
    policy += "\n"
    return policy
"""

'\n#input df: csv con informacion personal\n#output data_types: tipos de datos sensibles encontrados en cada columna\ndef get_data_types(df:pd.DataFrame)->dict[list]:\n  data_types = dict()\n  for column_name in df.columns:\n    data_type = send_prompt(data=get_data(df[column_name]))\n    sanitized_data_type = sanitize(data_type)\n    if sanitized_data_type not in data_types:\n      data_types[sanitized_data_type] = [column_name]\n    else:\n      data_types[sanitized_data_type].append(column_name)\n  return data_types\n\n#input regulation: Name of the regulation to use\n#output df_cumplimiento: df[Key Points, Description, Compliance Requirements]\ndef get_cumplimiento(regulation:str)->pd.DataFrame:\n  csv = send_prompt(mode=\'get_regulations\',regulation=regulation)\n  df_cumplimiento = pd.read_csv(StringIO(csv),sep=";",header=1)\n  for column_name in df_cumplimiento.columns:\n    df_cumplimiento = df_cumplimiento.rename(columns={df_cumplimiento.columns[0]: \'Key Points\', df_cumplimi

In [364]:
process0 = Process(
    "Customer Onboarding",
    ["Online Registration Form"],
    ["Name", "Email", "Address"],
    ["Providing Services"],
    ["Internal Sales Team", "Third-Party CRM"],
    ["Cloud Database"],
    ["5 Years"]
)

process1 = Process(
    "Customer Onboarding",
    ["Online Registration Form", "CRM System"],
    ["Name", "Email", "Address", "Phone Number"],
    ["Providing Services", "Marketing Communications"],
    ["Internal Sales Team", "Third-Party CRM", "Marketing Department"],
    ["Cloud Database", "CRM System"],
    ["5 Years", "Until Opt-Out"]
)

process2 = Process(
    "Product Development",
    ["Market Research Surveys", "Customer Feedback Forms", "Product Usage Data"],
    ["Age", "Gender", "Product Preferences"],
    ["Improving Products and Services", "Developing New Features"],
    ["Product Team", "Marketing Team", "Data Analytics Team"],
    ["Product Development Database", "Data Warehouse"],
    ["Indefinite", "Until Product End-of-Life"]
)

process3 = Process(
    "Order Fulfillment",
    ["E-commerce Platform", "Point-of-Sale System"],
    ["Name", "Shipping Address", "Payment Details", "Order History"],
    ["Delivering Purchased Products", "Processing Returns and Refunds"],
    ["Warehouse Team", "Shipping Providers", "Customer Service Team"],
    ["Order Management System", "Inventory Management System"],
    ["7 Years", "Until Account Closure"]
)

process4 = Process(
    "Employee Recruitment",
    ["Online Job Portals", "Employee Referrals", "Recruitment Agencies"],
    ["Name", "Email", "Resume", "Cover Letter", "References"],
    ["Hiring New Employees", "Building Talent Pipeline"],
    ["HR Department", "Hiring Managers", "Recruitment Agencies"],
    ["Applicant Tracking System", "HR Database"],
    ["3 Years", "Until Candidate Withdrawal"]
)

process5 = Process(
    "Invoice Processing",
    ["Email", "Physical Mail", "Vendor Portal"],
    ["Vendor Information", "Payment Details", "Invoice History"],
    ["Paying Vendors for Goods and Services", "Managing Vendor Relationships"],
    ["Accounts Payable Team", "Finance Department", "Procurement Team"],
    ["Accounting Software", "Vendor Management System"],
    ["7 Years", "In Accordance with Tax Regulations"]
)

In [390]:
class Regulation:
  def __init__(self,name=''):
    self.name:str = name
    self.df_cumplimiento:pd.DataFrame = None

  def set_cumplimiento(self,name:str):
    self.name = name
    self.df_cumplimiento = self.get_cumplimiento(self.name)

class PrivacyPolicy:
    def __init__(self):
        self.dpo = dpo = ''
        self.data_types:dict[list] = {}
        self.detected_data_types:dict[list] = {}
        self.gdpr_mapping:dict[list] = {}
        self.processes:dict[Process] = {}
        self.regulations:Regulation = None
        self.df_dataset:pd.DataFrame = None

    def get_dpo(self):
        return self.dpo

    def set_df_dataset(self,df_dataset:pd.DataFrame):
        self.df_dataset = df_dataset

    def set_detected_data_types(self):
        data_types = dict()
        for column_name in self.df_dataset.columns:
          data_type = send_prompt(data=get_data(self.df_dataset[column_name]))
          sanitized_data_type = sanitize(data_type)
          if sanitized_data_type not in data_types:
            data_types[sanitized_data_type] = [column_name]
          else:
            data_types[sanitized_data_type].append(column_name)
        self.detected_data_types = data_types

    def get_detected_data_types(self):
        return self.detected_data_types

    def get_all_data_types(self):
      all_data_types = set
      for process in self.processes:
        for data_type in self.processes[process].personal_data_types:
          all_data_types.add(data_type)
      return all_data_types

    def get_gdpr_mapping(self):
        return self.gdpr_mapping

    def add_regulation(self,regulation):
      self.regulations = self.get_cumplimiento(regulation)

    def add_process(self, process:Process):
      self.processes[process.name] = process
      #self.data_types[process.name] = process.personal_data_types
      self.gdpr_mapping[process.name] = process.purpose

    def get_cumplimiento(regulation:str)->pd.DataFrame:
      csv = send_prompt(mode='get_regulations',regulation=regulation)
      df_cumplimiento = pd.read_csv(StringIO(csv),sep=";",header=1)
      for column_name in df_cumplimiento.columns:
        df_cumplimiento = df_cumplimiento.rename(columns={df_cumplimiento.columns[0]: 'Key Points', df_cumplimiento.columns[1]: 'Description', df_cumplimiento.columns[2]:'Compliance Requirements'}, inplace=False)
      return df_cumplimiento

    def generate_privacy_policy(self, df_dataset:pd.DataFrame, lang:str)->str:
        """
        Generate a data privacy policy from a DataFrame containing privacy regulations.

        Parameters:
        self.regulations.columns (pd.DataFrame): DataFrame with columns like 'Regulation', 'Description', 'Key Points', 'Compliance Requirements'.
        df_dataset (pd.DataFrame): DataFrame with personal information.
        lang (str): Language code for translation.

        Returns:
        str: A formatted data privacy policy.
        """
        # Check for necessary columns in the DataFrame
        required_columns = ['Description', 'Key Points', 'Compliance Requirements']
        if not all(column in self.regulations.columns for column in required_columns):
            raise ValueError(f"DataFrame must contain the following columns: {required_columns}")

        # Start with an introductory section
        policy = "Data Privacy Policy\n"
        policy += "=" * 20 + "\n\n"
        policy += "This policy outlines our commitment to protecting personal data and complying with data privacy regulations.\n\n"

        # Generate sections for each regulation
        for index, row in self.regulations.columns.iterrows():
          if pd.isnull(row['Compliance Requirements']) and pd.isnull(row['Description']):
            policy += f"\nControl a realizar: {row['Key Points']}\n"
          else:
            if row['Key Points'].lower() != 'nan': policy += f"Key Point: {row['Key Points']}\n"
            if row['Compliance Requirements'].lower() != 'nan': policy += f"Compliance Requirements: {row['Compliance Requirements']}\n"
            if row['Description'].lower() != 'nan': policy += f"Description:\n{row['Description']}\n"
          policy += "\n" + "-" * 20 + "\n"

        policy += "Contact Information:\n"
        policy += "If you have any questions about this policy, please contact us at [your contact information].\n\n"

        policy += "Policy Review and Updates:\n"
        policy += "We may update this policy from time to time to reflect changes in our practices or legal requirements. Please check back periodically for updates.\n"

        #omegalul
        policy += policy_data_types(self.regulations.columns,df_dataset)
        if lang != 'english':
          policy = translate(policy,lang)
        return policy

    def policy_data_types(self, df_dataset:pd.DataFrame)->str:
        """
        Generate a data policy based on personal or sensitive information data types from a CSV file.

        Parameters:
        df_cumplimiento
        df_dataset

        Returns:
        str: A formatted data policy.
        """
        # Start generating the policy
        policy = "Data Privacy Policy\n"
        policy += "=" * 20 + "\n\n"
        policy += "This policy outlines our commitment to protecting personal and sensitive data types and complying with data privacy regulations.\n\n"

        # Add sections based on the data types
        policy += "Data Types Covered\n"
        policy += "-" * 20 + "\n"
        for data_type in self.detected_data_types:
            policy += f"- {data_type}\n"
        policy += "\n"
        return policy

    def get_processes(self):
        for process in self.processes:
            print(self.processes[process].name)
            print(self.processes[process].data_source)
            print(self.processes[process].personal_data_types)
            print(self.processes[process].purpose)
            print(self.processes[process].recipients)
            print(self.processes[process].storage_location)
            print(self.processes[process].retention_period)
            print()

class Process:
  def __init__(self, name:str, data_source:list[str], personal_data_types:list[str], purpose:list[str], recipients:list[str], storage_location:list[str], retention_period:list[str]):
    self.name = name
    self.data_source = data_source
    self.personal_data_types = personal_data_types
    self.purpose = purpose
    self.recipients = recipients
    self.storage_location = storage_location
    self.retention_period = retention_period

  def __str__(self):
    return f"Process Name: {self.name}\nData Source: {self.data_source}\nPersonal Data Types: {', '.join(self.personal_data_types)}\nPurpose: {self.purpose}\nRecipients: {', '.join(self.recipients)}\nStorage Location: {self.storage_location}\nRetention Period: {self.retention_period}"

In [391]:
dataset_filename = "dummy12.csv"
#df_dataset = pd.read_csv("/content/drive/MyDrive/11vo/poctestdata/dummy68.csv",sep=";")
df_dataset = pd.read_csv("/content/drive/MyDrive/11vo/poctestdata/"+dataset_filename,sep=",")
regulation = Regulation('HIPAA')
privacy_policy = PrivacyPolicy()
privacy_policy.add_process(process0)
privacy_policy.add_process(process1)
privacy_policy.add_process(process2)
privacy_policy.add_process(process3)
privacy_policy.add_process(process4)
privacy_policy.add_process(process5)
privacy_policy.get_processes()
privacy_policy.set_df_dataset(df_dataset)
privacy_policy.set_detected_data_types()
print(privacy_policy.get_all_data_types())
privacy_policy.get_detected_data_types()

Customer Onboarding
['Online Registration Form', 'CRM System']
['Name', 'Email', 'Address', 'Phone Number']
['Providing Services', 'Marketing Communications']
['Internal Sales Team', 'Third-Party CRM', 'Marketing Department']
['Cloud Database', 'CRM System']
['5 Years', 'Until Opt-Out']

Product Development
['Market Research Surveys', 'Customer Feedback Forms', 'Product Usage Data']
['Age', 'Gender', 'Product Preferences']
['Improving Products and Services', 'Developing New Features']
['Product Team', 'Marketing Team', 'Data Analytics Team']
['Product Development Database', 'Data Warehouse']
['Indefinite', 'Until Product End-of-Life']

Order Fulfillment
['E-commerce Platform', 'Point-of-Sale System']
['Name', 'Shipping Address', 'Payment Details', 'Order History']
['Delivering Purchased Products', 'Processing Returns and Refunds']
['Warehouse Team', 'Shipping Providers', 'Customer Service Team']
['Order Management System', 'Inventory Management System']
['7 Years', 'Until Account Closu

{'unknown': ['id'],
 'email address': ['email_address', 'personal_email'],
 'phone number': ['home_phone', 'fax_number', 'business_phone'],
 'city name': ['city'],
 'name': ['last_name', 'first_name'],
 'job title': ['job_title'],
 'country': ['country_region'],
 'career history': ['notes'],
 'credit card': ['company'],
 'address': ['address'],
 'location': ['state_province'],
 'zip code': ['zip_postal_code']}

The name and contact details of the controller and the processor acting on their behalf, as well as the processor or controller’s representative and DPO, if applicable;

A description of the various categories of users and data (including from third parties);

The categories of data recipients, including non-EU third-country recipients or international organizations;

The purpose of the processing activities;

Transfers of personal data to a third country and the identification of that third country or international organization, including documentation of suitable safeguards (where applicable);

Anticipated time limits for erasure of the various categories of data (where possible);

The technical and organizational security measures described in general terms (where possible).